Neural Net을 scratch로 구현해보고, 구현한 Neural Net의 부분 부분을 ``torch.nn``의 class와 method로 대체해가면서 어떠한 class/method가 scratch code의 어떠한 부분을 대체하는 지를 익힌다.

여러모로 **'02_Neural_Networks'**와 비슷한 내용을 담고있다.

## Table of Contents
### Part 0: Neural Net from Scratch
* MNIST  Data Setup
* Neural Net from Scratch (w/o torch.nn)

### Part 1: Simplifying Training Session with ``Torch.nn``
* **``torch.nn.functional``**
* **``nn.Module``**
* **``nn.Linear``**
* **``torch.optim``**
* **``torch.utils.data.TensorData``**
* **``torch.utils.data.DataLoader``**

### Part 2: Validation
#### Adding Validation
* Shuffling
* batch size
* ``model.train()`` & ``model.eval()``

### Part 3: Applying to CNN
* **Modeling CNN**
* **``nn.Sequential``**


### Summary (Important!)
#### How to Design NN
* Algortihms
* Using **``torch.nn``**

---

# Part 0: Neural Net from Scratch

## MNIST Data Setup

In [80]:
from pathlib import Path
import requests
import numpy as np
import pickle
import gzip
import torch

DATA_PATH= Path("data")
PATH= DATA_PATH / "mnist"

PATH.mkdir(parents= True, exist_ok= True)

URL= "https://github.com/pytorch/tutorials/raw/master/_static/"
FILENAME= "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
    content= requests.get(URL + FILENAME).content
    (PATH / FILENAME).open("wb").write(content)
    

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
        
x_train, y_train, x_valid, y_valid= map(
    torch.tensor,  (x_train,y_train, x_valid, y_valid)
)

n, c= x_train.shape
x_train,  x_train.shape, y_train.min(), y_train.max()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(),  y_train.max())

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) tensor([5, 0, 4,  ..., 8, 4, 8])
torch.Size([50000, 784])
tensor(0) tensor(9)


---
## Neural Net from Scratch (w/o torch.nn)
create Neural Net using only PyTorch Tensor operations(without ``torch.nn`` pacakge).

### Before Training
* weights, bias 초기화
* activation function
* model(linear layer)
    * linear layer에서 진행되는 computation을 정의해야 함.
    * m
* loss function
* accuracy function

### Training
매 epoch마다
* 학습 시킬 batch (x & y) 
* x로 prediction 진행
* x, y로 loss 계산
* ``.backward()``
* weight, bias update (with ``torch.no_grad`` b/c we shouldn't consider this process when calculating gradient)
* weight, bias grad_zero

#### Before Training

In [2]:
# weights & bias
import math

weights= torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias= torch.zeros(10, requires_grad= True)

In [3]:
# activation function
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

In [5]:
# model(linear layer)
def model(xb):
    return log_softmax(xb @ weights + bias)

bs= 64    # batch size
xb= x_train[:bs]
preds= model(xb)
print(preds[0], preds.shape)

tensor([-2.4852, -2.1391, -2.6954, -2.4873, -2.1909, -2.0923, -2.2463, -2.4395,
        -2.2207, -2.1934], grad_fn=<SelectBackward>) torch.Size([64, 10])


In [9]:
# loss function - negative log likelihood 
# 즉, negative log likelihood를 최소화 하는 문제
def nll(input_, target):
    return -input_[range(target.shape[0]), target].mean()
           # tensor_a[(0, 1), (3, 2)] returns tensor_a[0][3] and tensor_a[1][2]

loss_func= nll

yb= y_train[:bs]
print(loss_func(preds, yb))

tensor(2.4059, grad_fn=<NegBackward>)


<div class="alert alert-info"><h4>Question</h4><p> nll이 왜 저렇게 생겼지? </p></div>

In [8]:
# accruacy function
def accuracy(out, yb):
    preds= torch.argmax(out, dim=1)
    return (preds==yb).float().mean()

print(accuracy(preds, yb))

tensor(0.0781)


#### Training

In [32]:
lr= 0.5
epochs= 2
for epoch in range(epochs):
    for i in range((n-1) // bs + 1):
        start_i= i*bs
        end_i= start_i + bs
        xb= x_train[start_i:end_i]
        yb= y_train[start_i:end_i]
        pred= model(xb)
        loss= loss_func(pred, yb)
    
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * lr
        bias -= bias.grad * lr
        weights.grad.zero_()
        bias.grad.zero_()

In [38]:
print("loss: ", loss_func(model(xb), yb).item(), "\naccuracy: ",accuracy(model(xb), yb).item())

loss:  0.8000306487083435 
accuracy:  0.75


---
# Part 1: Simplifying Training Session
## ``torch.nn.functional``
``torch.nn.functional``은 ``torch.nn``의 모든 function들을 담고 있는 패키지이다(나머지는 모두 class를 담고있음).

functional의 function은
* loss function
* activation function
* other functions(pooling function, relu, ...)
등이 있다.

### Replace
negative log likelihood(loss function)과 log sogtmax(activation function) 대신, ``F.cross_entropy``를 사용 할 수 있다.

즉, **loss function**과 **activation function**을 **``torch.nn.functional``**의 method로 대체할 수 있다.

In [39]:
# original loss f, activation f, and model
'''
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)
    
def model(xb):
    return log_softmax(xb @ weights + bias)
    
def nll(input_, target):
    return -input_[range(target.shape[0]), target].mean()
'''

import torch.nn.functional as F

loss_func= F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [40]:
print("loss: ", loss_func(model(xb), yb).item(), "\naccuracy: ",accuracy(model(xb), yb).item())

loss:  0.800030529499054 
accuracy:  0.75


---

## ``nn.Module``
``nn.Module``을 통해 model Class를 정의하면 **weight/bias initalizing**과 **Forward Pass**를 쉽게 구현 할 수 있다.

* ``nn.Parameter``: module의 attribute로 ``nn.Paramter``를 assign한다면, 자동으로 모델의 **parameter**로 인식되며, ``.parameters()`` iterator를 통해 확인 할 수 있다. 이를 통해, weights와 bias를 따로따로 지정하지 않고 모델의 parameter들을 한번에 update할 수 있다. 특히, 여러개의 layer가 겹겹히 쌓인 model의 경우 훨씬 더 유용하다.
* ``model.zero_grad()``를 통해 각 parameter에 쌓인 gradient를 초기화 할 수 있다(매 iteration마다 gradient 초기화 필요)

### Replace
weight & bias 초기화, training 과정에서의 forward pass 대신, **``nn.Module``을 상속하는 Class를 정의할 수 있다.**

In [71]:
from torch import nn

# class - initiailzing weights & bias + defining forward pass
class Mnist_Logistics(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights= nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias= nn.Parameter(torch.zeros(10))
        
    def forward(self, xb):
        return xb @ self.weights + self.bias

In [72]:
model= Mnist_Logistics()

In [73]:
# training
def fit():
    for epoch in range(epochs):
        for i in range((n-1) // bs + 1):
            start_i= i * bs
            end_i = start_i + bs
            xb= x_train[start_i:end_i]
            yb= y_train[start_i:end_i]
            
            pred= model(xb)
            # we previously defined loss_func as F.cross_entropy
            loss= loss_func(pred, yb)
            
            # backward pass
            loss.backward()
            with torch.no_grad():
                # don't have to update weights and bias separately
                for p in model.parameters():
                    p -= p.grad * lr
            
            model.zero_grad()

In [74]:
fit()
print(loss_func(model(xb), yb))

tensor(0.0784, grad_fn=<NllLossBackward>)


---

## ``nn.Linear``
[``nn.Linear``](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)를 통해 weights와 bias를 initalize하고 forward pass 식을 간소화 할 수 있다.

당연한 얘기지만, linear layer기 때문에 ``nn.Linear``를 사용하는 것. 필요한 layer의 성격에 따라 다양한 method를 사용하거나, ``nn.Conv2d``와 같은 class를 사용 할 수 있다.

### Replace
다음의 과정들을 nn.Linear를 통해 구현 할 것이다.
* class의 ``__init__()``에서 weigths와 bias를 초기화 하는 과정 - bias도 자동으로 만들어준다.
* class의 ``forward()``의 forward pass 계산 식

In [65]:
class Mnist_Logistics(nn.Module):
    def __init__(self):
        super().__init__()
       #self.weights= nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
       #self.bias= nn.Parameter(torch.zeros(10))
        
        self.lin= nn.Linear(784, 10)
    
    def forward(self, xb):
      # return xb @ self.weights + self.bias
        return self.lin(xb)

In [67]:
model= Mnist_Logistics()
fit()
print(loss_func(model(xb), yb))

tensor(0.0811, grad_fn=<NllLossBackward>)


---
## ``torch.optim``
[``torch.optim``](https://pytorch.org/docs/stable/optim.html)을 통해 backward pass와 weights update과정을 간소화 할 수 있다.

``torch.optim``에는 다양한 optimizer가 담겨있다. 

optimizer를 생성 할 때에는 optimizer를 적용하고자 하는 model의 parameter를 ``model.parameters()``를 통해 전달해 준다(with learning rate).

각각의 optimizer는 다음과 같은 method를 수행 할 수 있다.
* ``.step()``:  ``.backward()``를 실행한 후, 그 결과를 바탕으로 **weights update**를 진행한다.
* ``.zero_grad()``: weights & bias에 쌓인 gradient를 초기화한다.

### Replace
다음의 과정들을 ``torch.optim``을 통해 구현 할 것이다.
* optimizer 생성
* fit()에서, backward pass이후 weight을 update

In [75]:
from torch import optim

# model과 함께 optimizer를 생성
def get_model():
    model= Mnist_Logistics()
    return model, optim.SGD(model.parameters(), lr= lr)

model, opt= get_model()

for epoch in range(epochs):
    for i in range((n-1) // bs + 1):
        start_i= i * bs
        end_i= start_i + bs
        xb= x_train[start_i:end_i]
        yb= y_train[start_i:end_i]
        pred= model(xb)
        loss= loss_func(pred, yb)
        
        loss.backward()
        # .backward()가 진행되었으므로, 이를 통해 weight update
        opt.step()
        opt.zero_grad()

In [76]:
print(loss_func(model(xb), yb))

tensor(0.0821, grad_fn=<NllLossBackward>)


---
## ``torch.utils.data.TensorDataset``

Pytorch는 ``DataSet`` Class가 있다. ``DataSet`` class는 1) ``__len__`` function과 2) ``__getitem__`` function (as a way of indexing into it)이 있는 모든 class를 일컫는다.

특히 이 중 [``torch.utils.data.TensorDataset``](https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset)의 경우, **Tensor**들을 **wrapping**하는 DataSet이다. 
 
### Replace

TensorDataset을 indexing하면, 각 tensor들이 first dimension을 기준으로 indexing되어 반환된다.
이는 NN 모델에서는 당연히, **x와 y를 같은 batch size로 indexing 할 때**에 사용된다.
1. 기존 indexing 방법

 xb= x_train[start_i: end_i]
, yb= y_train[start_i: end_i]


2. TensorDataset을 이용한 indexing 방법

 train_ds= TensorDataset(x_train, y_train)

 xb, yb= train_ds[start_i: end_i]

In [78]:
from torch.utils.data import TensorDataset

# TensorDataset 생성
train_ds= TensorDataset(x_train, y_train)

model, opt= get_model()

for epoch in range(epochs):
    for i in range((n-1) // bs + 1):
        '''start_i= i * bs
        end_i= start_i + bs
        xb= x_train[start_i:end_i]
        yb= y_train[start_i:end_i]'''
        # TensorDataset의 특성을 이용해 x와 y를 한번에 return 받는다.
        xb, yb= train_ds[i*bs:i*bs+bs]
        pred= model(xb)
        loss= loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
print(loss_func(model(xb), yb))

tensor(0.0812, grad_fn=<NllLossBackward>)


---
## ``torch.utils.data.DataLoader``

[``torch.utils.data.DataLoader``](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)는 batching을 하기 위해 특화된 PyTorch의 class. DataSet과 batch_size(1 if None)가 주어지면 그 size에 맞게 DataSet의 element들을 indexing하여 반환하는 iterator이다.

shuffle=True로 설정 할 시, dataset을 반환할 때 shuffle을 한 후 반환한다.
### Replace
batching 과정이 더더욱 간소화된다.
1. 기존 batching 방법

    for i in range((n-1) // bs +1)):
        xb, yb= train_ds[i*bs: i*bs+bs]
        pred= model(xb)

2. DataLoader를 이용한 batching 방법

    for xb, yb in train_dl:
        pred= model(xb)

In [81]:
from torch.utils.data import DataLoader

train_ds= TensorDataset(x_train, y_train)
# create DataLoader with TensorDataset and it's batch size
train_dl= DataLoader(train_ds, batch_size= bs)

model, opt= get_model()
for epoch in range(epochs):
    # DataLoader is iteratior
    for xb, yb in train_dl:
        pred= model(xb)
        loss= loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
print(loss_func(model(xb), yb))

tensor(0.0828, grad_fn=<NllLossBackward>)


---
Until here, using ``nn.functional``, ``nn.Module``, ``nn.Linear``, ``torch.optim``, ``TensorDataset``, ``DataLoader``, we simplified training session dramatically

---

# Part 2: Validation
## Add Validation

**Validation**은 Overfitting을 방지하기 위해 반드시 필요하다.

Neural Net에 validation 과정을 도입 할 때 고려 할 사항이 몇가지 있다.

### Shuffling
training data의 경우, overfitting을 방지하기 위해 batch를 생성 할 때 shuffle을 해주어야 한다.
그러나 validation data의 경우, **학습**하는 것이 아니므로, shuffle을 할 필요가 없다. 쓸데없는 연산을 줄이기 위해, validation batch를 생성 할 때 shuffle을 꺼주어야 한다.

### batch size
validation set의 경우 back propagation을 진행 할 필요가 없다(forward pass만을 진행하여 결과값만 구하면 된다). 따라서, 연산량이 훨씬 가볍기 때문에 batch size를 training 과정에서보다 늘려도 된다.

### ``model.train()`` & ``model.eval()``
``nn.BatchNorm2d``나 ``nn.Dropout``과 같은 layer의 경우, training 과정과 validation 과정에서의 작동 방식이 다르기 때문에 현재 진행하고자 하는 step을 설명해주어야 한다.

따라서, training 전에는  ``model.train()``을, validation 전에는 ``model.eval()``을 선언해주어야 한다.

#### Shuffling & Batch size

<div class="alert alert-info"><h4>Question</h4><p> Why do we have to batch the validation set? Can't we just compute it altogether? </p></div>

In [82]:
train_ds= TensorDataset(x_train, y_train)
train_dl= DataLoader(train_ds, batch_size= bs)

valid_ds= TensorDataset(x_valid, y_valid)
valid_dl= DataLoader(valid_ds, batch_size= bs*2)

#### model.train() & model.eval()

In [84]:
model, opt= get_model()
epochs= 5

for epoch in range(epochs):
    model.train()
    for xb, yb in train_dl:
        pred= model(xb)
        loss= loss_func(pred, yb)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    model.eval()
    with torch.no_grad():
        valid_loss= sum(loss_func(model(xb), yb) for (xb, yb) in valid_dl)
        
    print("epoch: %d, loss: %.4f" % (epoch+1, (valid_loss.item() / len(valid_dl))))

epoch: 1, loss: 0.3099
epoch: 2, loss: 0.2902
epoch: 3, loss: 0.2833
epoch: 4, loss: 0.2800
epoch: 5, loss: 0.2782


---
### loss의 함수화
loss를 계산하는 과정은 training에서도 validation에서도 사용되므로, 하나의 함수를 정의해 코드의 길이를 줄일 수 있다.

In [88]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss= loss_func(model(xb), yb)
    
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    
    return loss.item(), len(xb)

In [89]:
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size= bs, shuffle= True),
        DataLoader(valid_ds, batch_size= bs*2)
    )

In [90]:
import numpy as np

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)
        
        model.eval()
        
        with torch.no_grad():
            losses, nums= zip(
                *[loss_batch(model, loss_func, xb, yb) for (xb, yb) in valid_dl]
            )
        val_loss= np.sum(np.multiply(losses, nums)) / np.sum(nums)
        
        print("epoch: %d, validaiton loss: %.4f" % (epoch+1, val_loss))

In [91]:
train_dl, valid_dl= get_data(train_ds, valid_ds, bs)
model, opt= get_model()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

epoch: 1, validaiton loss: 0.4107
epoch: 2, validaiton loss: 0.2994
epoch: 3, validaiton loss: 0.3944
epoch: 4, validaiton loss: 0.2811
epoch: 5, validaiton loss: 0.2838


---
# Part 3: Applying to CNN

## Modeling CNN
위에서 정의한 함수들(``fit()``과 그 안에 투입되는 함수들)의 경우, 특정 Neural Network 모델을 염두해 두고 만든 것이 아닌, 보편적으로 적용 될 수 있는 함수이다.

따라서, 적절한 model을 fit 함수에 집어넣을 경우 다양한 Neural Network을 구현 할 수 있다

In [94]:
class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1)

    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.conv3(xb))
        xb = F.avg_pool2d(xb, 4)
        return xb.view(-1, xb.size(1))

lr = 0.1

In [95]:
model= Mnist_CNN()
opt= optim.SGD(model.parameters(), lr= lr, momentum= 0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

epoch: 1, validaiton loss: 0.3867
epoch: 2, validaiton loss: 0.2670
epoch: 3, validaiton loss: 0.2067
epoch: 4, validaiton loss: 0.1875
epoch: 5, validaiton loss: 0.1818


---
## ``nn.Sequential``
``nn.Sequential``은 안에 담긴 module들을 순차적으로 실행하는 class이다. 일종의 **pipeline**을 만드는 셈

model에 function을 사용하고 싶어도 **layer**의 형태를 띄어야 하므로, 아래와 같은 일종의 **가짜 layer를 만들고, 그 안에 넣을 함수도 만든다.**

In [96]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func= func
        
    def forward(self, x):
        return self.func(x)

In [97]:
def preprocess(x):
    return x.view(-1, 1, 28, 28)

In [98]:
model= nn.Sequential(
    Lambda(preprocess),
    nn.Conv2d(1, 16, kernel_size= 3, stride= 2, padding= 1),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size= 3, stride= 2, padding= 1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size= 3, stride= 2, padding= 1),
    nn.ReLU(),
    nn.AvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

opt= optim.SGD(model.parameters(), lr= lr, momentum= 0.9)

fit(epochs, model, loss_func, opt, train_dl, valid_dl)

epoch: 1, validaiton loss: 0.3268
epoch: 2, validaiton loss: 0.2670
epoch: 3, validaiton loss: 0.2085
epoch: 4, validaiton loss: 0.2047
epoch: 5, validaiton loss: 0.1485


---
---
# Summary (Important!)

## How To Design NN
I am Supposing we are building NN with single linear layer.

### Before Training
* Initialize **weights** and **bias**
    * by **subclass**ing *``nn.Module``* and  *``nn.Linear``*
* Define **Activation Function**
    * by *``torch.nn.functional``*
* Design **Model** - computation inside layer
    * by **subclass**ing *``nn.Module``* and *``nn.Linear``*
* Define **Loss Function**
    * by *``torch.nn.functional``*

### During Training
#### In every  epoch,
* Make **batch** for training(x_batch, y_batch)
    * by *``torch.utils.data.TensorDataset``* and *``torch.utils.data.DataLoader``*
* Make **prediction** with x
    * by **subclass**ing *``nn.Module``*
* Compute **loss**
    * by *``torch.nn.functional``*
* **backward pass**
* **Update** weights and bias -  with ``torch.no_grad``
    * by **subclass**ing *``nn.Module``* and *``torch.optim``*
* **Zero** gradients of weights and bias
    * by **subclass**ing *``nn.Module``* and *``torch.optim``*

#### Validation
* Make **batch** for validation
    * by *``torch.utils.data.TensorDataset``* and *``torch.utils.data.DataLoader``*
* Compute **loss**
    * by *``torch.nn.functional``*